In [ ]:
import torch                          # Le cœur de PyTorch : tensors, calculs, GPU
import torch.nn as nn                 # Contient toutes les couches du réseau (Conv2D, Linear, etc.)
import torch.optim as optim           # Optimiseurs pour entraîner le modèle (Adam, SGD, etc.)
from torch.utils.data import DataLoader  # Pour gérer le batching et le shuffle des datasets
from torch.utils.data import random_split
from torchvision import datasets, transforms, models  # Outils pour datasets, transformations et modèles pré-entraînés


import numpy as np                    # Manipulation de tableaux, conversion images → tensors
import pandas as pd                   # Pour lire et écrire le fichier test_template.csv
import os                             # Gestion des fichiers et dossiers

# -----------------------------
# Librairies pour la visualisation
# -----------------------------
import matplotlib.pyplot as plt       # Visualiser les images et tracer des courbes

# -----------------------------
# Librairies pour la vision par ordinateur
# -----------------------------
import cv2                            # Capturer la webcam et détecter les visages
from PIL import Image 
from skimage import io
import matplotlib.pyplot as plt
import os, random

In [ ]:
#observations des images
emotions = ['neutral', 'angry', 'fear', 'happy', 'sad', 'surprise', 'disgust']
emotion = 'neutral'
x = random.randint(0, 6)
path = f'data/dataTrain/train/{emotion}/'
img = io.imread(os.path.join(path, random.choice(os.listdir(path))))
plt.imshow(img, cmap='gray')
plt.title(emotion)
# plt.show()

In [ ]:
import os

for emotion in os.listdir('data/dataTrain/train'):
    print(emotion, ":", len(os.listdir(os.path.join('data/dataTrain/train', emotion))))


In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels = 3),            # images en niveaux de gris
    transforms.Resize((224, 224)), # redimensionne à 48x48

# data augmentation. eviter apprendre par coeur (chaque epoch change les images pour paraitre différement pour le modele)
    transforms.RandomHorizontalFlip(),           # augmentation horizontale
    transforms.ColorJitter(brightness=0.2, contrast=0.2),




    transforms.ToTensor(),             # convertit en tenseur [0,1]
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)) # normalise entre -1 et 1
])

train_data = datasets.ImageFolder("data/dataTrain/train", transform=transform)
test_data  = datasets.ImageFolder("data/dataTest",  transform=transform)


In [ ]:
#80% des données d'entrainement pour l'entrainement et 20% pour validation
train_size = int (0.8 * len(train_data))
validation_size = len(train_data) - train_size
train_dataset, validation_dataset = random_split(train_data, [train_size, validation_size])
test_loader  = DataLoader(test_data, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=True) #batch size 128 plus rapide

In [ ]:
# --- Modèle : ResNet18 fine-tuning ---
modeleEmotions = models.resnet18(pretrained=True)

# Dé-geler uniquement le dernier bloc convolutionnel + fc
for name, param in modeleEmotions.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

nbEmotions = 7
modeleEmotions.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(512, nbEmotions)
)


print(modeleEmotions) # Modèle de classification des émotions


In [ ]:
#loss function on ne veut pas geler toutes les couches. Sinon, le modele s'ameliore trop lentement
def get_device():
    # NVIDIA GPU (Windows/Linux, parfois Mac via eGPU mais rare)
    if torch.cuda.is_available():
        return torch.device("cuda")
    # Apple Silicon (Mac M1/M2/M3)
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    # Fallback
    return torch.device("cpu")
print(torch.__version__)
device = get_device()
print("Using device:", device)
modeleEmotions = modeleEmotions.to(device)

# --- Optimiseur et scheduler ---
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, modeleEmotions.parameters()),
    lr=1e-4
)

# Scheduler qui réduit le learning rate. eviter stagnation
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',          # surveille la val_loss
    factor=0.5,          # réduit LR de moitié
    patience=1,          # attend 1 epoch avant de réduire
)

criterion = nn.CrossEntropyLoss()

In [ ]:
#sanity check
images, labels = next(iter(train_loader))
images, labels = images.to(device), labels.to(device)

outputs = modeleEmotions(images)
print("outputs shape:", outputs.shape)
print("labels shape:", labels.shape)
print("labels dtype:", labels.dtype)
print("labels min/max:", labels.min().item(), labels.max().item())  # doit être 0..6

In [ ]:
num_epochs = 20 # 20 epoch de base pour chaque modification

#fonction principace d'entraiment et de validation
def run_one_epoch(model, loader, training: bool):
    if training:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    correct = 0
    total = 0

    with torch.set_grad_enabled(training):
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            if training:
                optimizer.zero_grad()

            outputs = model(images)              # (batch, nbEmotions)
            loss = criterion(outputs, labels)

            if training:
                loss.backward()
                optimizer.step()

            total_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss / total, correct / total


# --- Boucle principale ---
for epoch in range(num_epochs):
    train_loss, train_acc = run_one_epoch(modeleEmotions, train_loader, training=True)
    val_loss, val_acc = run_one_epoch(modeleEmotions, validation_loader, training=False)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train loss {train_loss:.4f} acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f} acc {val_acc:.4f}")

